In [1]:
import numpy as np
import random
from collections import defaultdict

In [48]:
class IndependentCascade():
    def __init__(self):
        self.preds = defaultdict(dict)
        self.succs = defaultdict(dict)
    def createGraph(self,episodes):
        for ep in episodes:
            succs = [ep[ep[:,1] > ep[i,1]][:,0] for i in xrange(len(ep))]
            for i,succ in enumerate(succs):
                for s in succ:
                    s = int(s)
                    r = random.random() / 20
                    self.succs[i][s] = r
                    self.preds[s][i] = r
    def fit(self, episodes):
        self.episodes = episodes
        p = [{} * len(episodes)]
        while True:
            for d in xrange(len(episodes)):
                episode = episodes[d]
                for event in episode:
                    u = event[0]
                    ut = event[1]
                    p[d][u] = 1
                    for v in self.preds[u].keys():
                        [episode[:,1] == ut - 1]
                        
            
    def inference(self, s0):
        #infected = np.zeros((len(self.succs)+1), bool)
        infected = defaultdict(bool)
        s = []
        s.append(s0)
        t = 1
        stop = False
        while s[t-1]:
            s.append([])
            for i in s[t-1]:
                for j in self.succs[i].keys():
                    if (not infected[j]) and (random.random() < self.succs[i][j]):
                        infected[j] = True
                        s[t].append(j)
            t = t + 1
        return s, infected
    def predict(self, s0, nIter):
        #infected = np.zeros((len(self.succs)+1), int)
        suminfected = defaultdict(float)
        for i in xrange(nIter):
            _, infected = self.inference(s0)
            for j in infected.keys():
                suminfected[j] = suminfected[j] + infected[j]
        for j in xrange(len(suminfected)):
            suminfected[j] = suminfected[j] / nIter
        return suminfected


In [43]:
def loadEpisodes(datafile="./train"):
    f = open(datafile)
    episodes = []
    for episode in f.readlines():
        ep = episode[:-3]
        ep = np.array([ept.split(":") for ept in ep.split(";")], float)
        episodes.append(ep[ep[:,1].argsort()])
    return episodes

In [44]:
episodes =  loadEpisodes()

In [49]:
ic = IndependentCascade()
ic.createGraph(episodes)

In [50]:
ic.inference([1,2])

([[1, 2],
  [84, 87, 97, 18, 24, 72],
  [27, 52, 63, 32, 78],
  [22, 26, 28, 54, 73, 12, 15, 55, 33],
  [89, 16, 69, 35, 41, 50, 80, 98, 99],
  [37, 71, 11, 77],
  [0, 1, 30, 31, 68, 74, 13, 38, 49, 59, 62, 86],
  [14, 44, 76, 96, 34, 56, 36, 8, 61, 67, 58],
  [9, 94, 79, 19, 93, 40],
  [42, 82, 53, 88],
  []],
 defaultdict(bool,
             {0: True,
              1: True,
              2: False,
              3: False,
              4: False,
              5: False,
              6: False,
              7: False,
              8: True,
              9: True,
              10: False,
              11: True,
              12: True,
              13: True,
              14: True,
              15: True,
              16: True,
              17: False,
              18: True,
              19: True,
              20: False,
              21: False,
              22: True,
              23: False,
              24: True,
              25: False,
              26: True,
              27: 

In [54]:
ic.predict([1,2],10000)

defaultdict(float,
            {0: 0.353,
             1: 0.4795,
             2: 0.4002,
             3: 0.374,
             4: 0.4215,
             5: 0.4157,
             6: 0.357,
             7: 0.4005,
             8: 0.4346,
             9: 0.3971,
             10: 0.4147,
             11: 0.4351,
             12: 0.397,
             13: 0.3449,
             14: 0.3934,
             15: 0.4766,
             16: 0.434,
             17: 0.3672,
             18: 0.4049,
             19: 0.401,
             20: 0.429,
             21: 0.3607,
             22: 0.4271,
             23: 0.4632,
             24: 0.3811,
             25: 0.4212,
             26: 0.4307,
             27: 0.3836,
             28: 0.4415,
             29: 0.4464,
             30: 0.3977,
             31: 0.4592,
             32: 0.4353,
             33: 0.4035,
             34: 0.4474,
             35: 0.4262,
             36: 0.431,
             37: 0.4266,
             38: 0.4489,
             39: 0.3807,

In [55]:
episodes[0]

array([[ 26.,   1.],
       [ 56.,   1.],
       [ 93.,   2.],
       [ 41.,   2.],
       [ 42.,   2.],
       [ 89.,   3.],
       [  8.,   3.],
       [ 43.,   4.],
       [  3.,   4.],
       [ 73.,   4.],
       [ 86.,   5.],
       [ 98.,   5.],
       [ 61.,   5.],
       [ 84.,   5.],
       [ 48.,   6.],
       [ 47.,   6.],
       [ 50.,   7.],
       [ 20.,   7.],
       [ 12.,   7.],
       [ 55.,   7.],
       [ 96.,   7.],
       [  5.,   8.],
       [ 52.,   8.],
       [ 34.,   9.],
       [ 54.,   9.],
       [  4.,  10.],
       [ 82.,  11.]])